In [ ]:
import numpy as np
import os, sys
from skimage import io
from matplotlib.colors import ListedColormap
from matplotlib.cm import hsv
import math
from tqdm import tqdm
from matplotlib import pyplot as plt
from matplotlib import colors
from pylab import cm

from pprint import pprint
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(PATH)
from utilities.utilities_cvat_neuroglancer import NumpyToNeuroglancer, get_cpus, get_segment_ids


In [ ]:
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(PATH)
from utilities.sqlcontroller import SqlController
from utilities.file_location import DATA_PATH, ROOT_DIR

In [ ]:
INPUT = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/X/preps/CH2/full_aligned'
files = sorted(os.listdir(INPUT))
midpoint = len(files) // 2
midfilepath = os.path.join(INPUT, files[midpoint])
limit = 1
files = files[midpoint-limit:midpoint+limit]
#files = [f for i,f in enumerate(files) if i % 10 == 0]
midfile = io.imread(midfilepath)
height, width = midfile.shape
data_type = np.uint8
volume_size = (width, height, len(files))
volume = np.zeros((volume_size), dtype=data_type)

In [ ]:
ids = [int(i) for i in np.unique(volume[:])]

In [ ]:
# use skimage
for i, f in enumerate(tqdm(files)):
    infile = os.path.join(INPUT, f)
    img = io.imread(infile)
    img = img.T
    volume[:,:,i] = img.reshape(img.shape[0], img.shape[1])

In [ ]:
labels = [[v-8,v-1] for v in range(9,256,8)]
arr = np.copy(midfile)
for label in labels:
    mask = (arr >= label[0]) & (arr <= label[1])
    arr[mask] = label[1]
arr[arr > 248] = 255        
midfile = arr.T

In [ ]:
print(f'midfile shape: {midfile.shape} dtype: {midfile.dtype} counts {np.unique(midfile, return_counts=True)}')

In [ ]:
#volume[volume > 0] = 255
#volume[volume < 1] = 0
flat = midfile.flatten()
fig = plt.figure()
plt.rcParams['figure.figsize'] = [10, 6]
plt.hist(flat, bins=35, color='b')
plt.style.use('ggplot')
plt.yscale('log')
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title(f' {volume.dtype}')

In [ ]:
segment_properties = get_segment_ids(midfile)

In [ ]:
info = {
    "@type": "neuroglancer_segment_properties",
    "inline": {
        "ids": [str(number) for number, label in segment_properties],
        "properties": [{
            "id": "label",
            "type": "label",
            "values": [str(label) for number, label in segment_properties]
        }]
    }
}
pprint(info)

In [ ]:
def get_hex_from_id(id, colormap='coolwarm'):
    cmap = cm.get_cmap(colormap, 255)
    rgba = cmap(id)
    return colors.rgb2hex(rgba)

In [ ]:
get_hex_from_id(0, 'gray')

In [ ]:
ids = [  8,  16,  24,  32,  40,  48,  56,  64,  72,  80,  88,  96,
       104, 112, 120, 128, 136, 144, 152, 160, 168, 176, 184, 192, 200,
       208, 216, 224, 232, 240, 248, 255]
cmap = cm.get_cmap('seismic', 255)    # PiYG
print("\"segmentColors\": {")
for id in ids:
    rgba = cmap(id)
    # rgb2hex accepts rgb or rgba
    row = f'\t"{id}": "{colors.rgb2hex(rgba)}",'
    print(row)
print("}")

In [ ]:
arr = np.rot90(volume,axes=(0, 1))
#arr = np.rot90(arr, 3)
arr = np.flip(arr, axis=0)
#arr = volume.copy()
print(arr.shape)
start = 0
end = arr.shape[2]
nrows = end - start
ncols = 1
plt.style.use('classic')
for i in range(start, end, 1):
    plt.title('Z {}'.format(i))
    plt.imshow(arr[:,:,i], cmap="seismic")
    plt.show()
    if i > 1:
        break